# Rayleigh-Bénard Convection in a plane layer



In [1]:
import petsc4py
from petsc4py import PETSc

import underworld3 as uw
from underworld3.systems import Stokes
from underworld3 import function

import numpy as np
import sympy
import os

# to fix trame issue
import nest_asyncio
nest_asyncio.apply()

[esdhcp-137.anu.edu.au:51653] shmem: mmap: an error occurred while determining whether or not /var/folders/tx/95gr762j29z4tt5d1dnqlgth0000gn/T//ompi.esdhcp-137.501/jf.0/1340342272/sm_segment.esdhcp-137.501.4fe40000.0 could be created.


In [2]:
# The problem setup 

width=6

# Parameters that define the notebook
# These can be set when launching the script as
# mpirun python3 scriptname -uw_resolution=0.1 etc

rayleigh_number_exponent = uw.options.getReal("rayleigh_no_expt",6) 
resolution = uw.options.getInt("model_resolution", default=8)
max_steps = uw.options.getInt("max_steps", default=50)
restart_step = uw.options.getInt("restart_step", default=-1)

# How that works

rayleigh_number = pow(10,rayleigh_number_exponent)
expt_name = f"Ra1e{rayleigh_number_exponent}_res{resolution}"
output_dir = os.path.join("output","cartesian",f"Ra1e{rayleigh_number_exponent}")

os.makedirs(output_dir, exist_ok=True)


In [3]:
## Set up the mesh geometry / discretisation

meshbox = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0.0,0.0),
    maxCoords=(width,1.0), 
    cellSize=1/resolution, 
    degree=1, 
    qdegree=3,
    regular=True,
)

meshbox.return_coords_to_bounds = None
meshbox.dm.view()

x,y  = meshbox.CoordinateSystem.X
x_vector = meshbox.CoordinateSystem.unit_e_0
y_vector = meshbox.CoordinateSystem.unit_e_1


DM Object: uw_.meshes/uw_simplexbox_minC(0.0, 0.0)_maxC(6, 1.0)_csize0.125_regTrue.msh 1 MPI process
  type: plex
uw_.meshes/uw_simplexbox_minC(0.0, 0.0)_maxC(6, 1.0)_csize0.125_regTrue.msh in 2 dimensions:
  Number of 0-cells per rank: 441
  Number of 1-cells per rank: 1208
  Number of 2-cells per rank: 768
Labels:
  depth: 3 strata with value/size (0 (441), 1 (1208), 2 (768))
  All_Boundaries: 1 strata with value/size (1001 (112))
  Bottom: 1 strata with value/size (11 (95))
  Elements: 1 strata with value/size (99999 (1097))
  Left: 1 strata with value/size (14 (15))
  Right: 1 strata with value/size (13 (15))
  Top: 1 strata with value/size (12 (95))
  celltype: 3 strata with value/size (0 (441), 1 (1208), 3 (768))
  UW_Boundaries: 5 strata with value/size (11 (95), 12 (95), 13 (15), 14 (15), 1001 (112))


In [5]:
v_soln = uw.discretisation.MeshVariable("U", meshbox, meshbox.dim, degree=2)
p_soln = uw.discretisation.MeshVariable("P", meshbox, 1, degree=1)
t_soln = uw.discretisation.MeshVariable("T", meshbox, 1, degree=3)

In [6]:
# passive_swarm = uw.swarm.Swarm(mesh=meshbox)
# passive_swarm.populate(fill_param=3)

In [7]:
# Create solver to solver the momentum equation (Stokes flow)

stokes = Stokes(
    meshbox, 
    velocityField=v_soln, 
    pressureField=p_soln, 
    solver_name="stokes",
)

stokes.tolerance = 0.0001

stokes.constitutive_model = uw.constitutive_models.ViscousFlowModel
stokes.constitutive_model.Parameters.viscosity = 1

# Penalise flow crossing the boundary

# stokes.add_natural_bc(10*rayleigh_number * v_soln.sym[1] * y_vector, "Top")
# stokes.add_natural_bc(10*rayleigh_number * v_soln.sym[1] * y_vector, "Bottom")
# stokes.add_natural_bc(10*rayleigh_number * v_soln.sym[0] * x_vector, "Left")
# stokes.add_natural_bc(10*rayleigh_number * v_soln.sym[0] * x_vector, "Right")

stokes.add_dirichlet_bc((None, 0.0), "Bottom")
stokes.add_dirichlet_bc((None, 0.0), "Top")
stokes.add_dirichlet_bc((0.0, None), "Left")
stokes.add_dirichlet_bc((0.0, None), "Right")

stokes.bodyforce = y_vector * rayleigh_number * t_soln.sym[0]

In [8]:
# Create solver for the energy equation (Advection-Diffusion of temperature)

adv_diff = uw.systems.AdvDiffusion(
    meshbox,
    u_Field=t_soln,
    V_fn=v_soln,
    solver_name="adv_diff",
    order=2,
)

adv_diff.constitutive_model = uw.constitutive_models.DiffusionModel
adv_diff.constitutive_model.Parameters.diffusivity = 1

## Boundary conditions for this solver

adv_diff.add_dirichlet_bc(1.0, "Bottom")
adv_diff.add_dirichlet_bc(0.0, "Top")

In [9]:
# The advection / diffusion equation is an initial value problem
# We set this up to have 

init_t = 0.25 * sympy.cos(5 * sympy.pi * x) * sympy.sin(sympy.pi * y) + (1-y)

with meshbox.access(t_soln):
    t_soln.data[...] = uw.function.evaluate(init_t, t_soln.coords).reshape(-1, 1)

if restart_step != -1:
    print(f"Reading step {restart_step}")
    t_soln.read_timestep(data_filename=f"{expt_name}",
                         data_name="T",
                         index=restart_step,
                         outputPath=output_dir)


In [10]:
# Solution strategy: solve for the velocity field, then update the temperature field.
# First we check this works for a tiny timestep, later we will repeat this to 
# model the passage of time.

stokes.solve()
adv_diff.solve(timestep=0.00001 * stokes.estimate_dt())

In [11]:
timestep = 0
elapsed_time=0.0

In [12]:
# Convection model / update in time

output = os.path.join(output_dir, expt_name)

for step in range(0, max_steps):
    stokes.solve()
    delta_t = 2.0 * stokes.estimate_dt()
    adv_diff.solve(timestep=delta_t)

    # stats then loop
    tstats = t_soln.stats()

    if uw.mpi.rank == 0:
        print(f"Timestep {timestep}, dt {delta_t}, t {elapsed_time}")
    
        meshbox.write_timestep(filename=f"{expt_name}",
                            index=timestep,
                            outputPath=output_dir,
                            meshVars=[v_soln, p_soln, t_soln],
                        )

    timestep += 1
    elapsed_time += delta_t


Timestep 0, dt 0.0001293870661420769, t 0.0
Timestep 1, dt 9.754585217777986e-05, t 0.0001293870661420769
Timestep 2, dt 8.454860805146877e-05, t 0.00022693291831985676
Timestep 3, dt 7.988965190222662e-05, t 0.00031148152637132555
Timestep 4, dt 7.958956447695483e-05, t 0.0003913711782735522
Timestep 5, dt 8.047738299761243e-05, t 0.000470960742750507
Timestep 6, dt 7.967446591690691e-05, t 0.0005514381257481194
Timestep 7, dt 8.304606144591268e-05, t 0.0006311125916650263
Timestep 8, dt 9.352691722507855e-05, t 0.0007141586531109389
Timestep 9, dt 0.00011008758654951942, t 0.0008076855703360175
Timestep 10, dt 0.0001273969995148167, t 0.0009177731568855369
Timestep 11, dt 0.0001503077189881397, t 0.0010451701564003536
Timestep 12, dt 0.00017188400291217064, t 0.0011954778753884934
Timestep 13, dt 0.00021293636331411653, t 0.001367361878300664
Timestep 14, dt 0.0002478726810813879, t 0.0015802982416147806
Timestep 15, dt 0.00025841464334553244, t 0.0018281709226961687
Timestep 16, dt 

In [17]:
if uw.mpi.size == 1:
    
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(meshbox)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym)
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)

    pvmesh_v = vis.meshVariable_to_pv_mesh_object(v_soln, alpha=None)
    pvmesh_v.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh_v, v_soln.sym)

    pvmesh_t = vis.meshVariable_to_pv_mesh_object(t_soln, alpha=None)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, t_soln.sym)

    # point sources at cell centres
    cpoints = pvmesh.cell_centers().points
    cpoint_cloud = pv.PolyData(cpoints)



    pvstream = pvmesh.streamlines_from_source(
            cpoint_cloud,
            vectors="V",
            integrator_type=2,
            integration_direction="both",
            compute_vorticity=False,
            max_steps=1000,
            max_time=0.25,
            initial_step_length=0.01,
            surface_streamlines=True,
            )

    pl = pv.Plotter(window_size=(750, 750))

    pl.add_mesh(
        pvmesh_t,
        cmap="RdBu_r",
        edge_color="Black",
        edge_opacity=0.1,
        show_edges=True,
        scalars="T",
        use_transparency=False,
        opacity=1.0,
        show_scalar_bar=False
    )

    pl.add_mesh(pvstream, opacity=0.5, 
                show_scalar_bar=False)

    pl.show(cpos="xy")

Widget(value='<iframe src="http://localhost:61342/index.html?ui=P_0x320cc39d0_4&reconnect=auto" class="pyvista…